In [28]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load
school_data = Path("Resources/schools_complete.csv")
student_data = Path("Resources/students_complete.csv")

# Read School and Student Data Files and store into Pandas DataFrames
school_data_df = pd.read_csv(school_data)
student_data_df = pd.read_csv(student_data)

# Combine the school and student data files into one single dataset.  
school_data_merged = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])

#school_data_merged.head(3)

In [29]:
# Local Government Area (LGA) Summary
# Perform the necessary calculations and create the local government area's key metrics in a DataFrame.

Totalnumber_of_uniqueschools = len(school_data_merged["school_name"].unique())
#Totalnumber_of_uniqueschools

In [30]:
Total_students = len(school_data_merged["Student ID"])
#Total_students

In [31]:
Total_budget = sum(school_data_merged["budget"].unique())
#Total_budget

In [32]:
Average_maths_score = school_data_merged["maths_score"].mean()
#Average_maths_score

In [33]:
Average_reading_score = school_data_merged["reading_score"].mean()
#Average_reading_score 

In [34]:
num_students_passing_maths = len((school_data_merged.loc[school_data_merged["maths_score"]>=50]))
num_students_passing_maths
percentage_students_passing_maths = (num_students_passing_maths/Total_students)*100
#percentage_students_passing_maths

In [35]:
num_students_passing_reading = len((school_data_merged.loc[school_data_merged["reading_score"]>=50]))
num_students_passing_reading
percentage_students_passing_reading = (num_students_passing_reading/Total_students)*100
#percentage_students_passing_reading

In [36]:
# filter students with overall passing grades first
overall_passing_df = school_data_merged.loc[(school_data_merged["maths_score"]>=50) & (
    school_data_merged["reading_score"]>=50), :]
overall_passing_df
# calculate the % of overall passing
percentage_students_passing_overall = (len(overall_passing_df)/Total_students)*100
#percentage_students_passing_overall

In [37]:
# Place all of the calculation into a summary DataFrame
area_summary = pd.DataFrame({"Total Schools": [Totalnumber_of_uniqueschools],
                              "Total Students": Total_students,
                              "Total Budget": Total_budget,
                              "Average Maths Score": Average_maths_score,
                              "Average Reading Score": Average_reading_score,
                              "% Passing Math": percentage_students_passing_maths, 
                              "% Passing Reading": percentage_students_passing_reading,
                              "% Overall Passing": percentage_students_passing_overall
                                     })
area_summary

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,70.338192,69.980138,86.078632,84.426857,72.808272


In [38]:
# School Summary

# Value counts and checked the student counts are the same as "size" in the school file

school_student_counts = school_data_merged["school_name"].value_counts()
# school_student_counts.head(14)

# use the dataset "schools_complete" as the base file for school summary
# extract the data required for "School Summary" from the file "schools_complete"
school_summary_df = school_data_df[["school_name","type","size","budget"]]
#school_summary_df.head()
# Add a new column that calculates the per student budget
school_summary_df["Per Student Budget"] = \
        school_summary_df["budget"] / \
        school_summary_df["size"]
#school_summary_df.head()

In [39]:
# Calculate average math scores using groupby and convert it into DataFrame
Average_maths_score_schools_df = pd.DataFrame(school_data_merged.groupby("school_name")["maths_score"].mean())
#print(Average_maths_score_schools_df)
# Merge this dataframe to the school summary dataset
merged1_school_summary_df = pd.merge(school_summary_df, Average_maths_score_schools_df, on="school_name", how="outer")
#merged_school_summary_df.head()

In [40]:
# Calculate average reading scores using groupby and convert it into DataFrame
Average_reading_score_schools_df = pd.DataFrame(school_data_merged.groupby("school_name")["reading_score"].mean())
# print(Average_reading_score_schools)
# Merge this dataframe to the school summary dataset merged with maths score
merged2_school_summary_df = pd.merge(merged1_school_summary_df, Average_reading_score_schools_df, on="school_name", how="outer")
# merged2_school_summary_df.head()

In [41]:
# Calculate the number of passing maths by school and convert it to dataframe
students_passing_math = school_data_merged.loc[school_data_merged["maths_score"]>=50]
#students_passing_math.head()

school_num_student_passing_maths_df = pd.DataFrame(students_passing_math.groupby("school_name")["student_name"].count())
#school_num_student_passing_maths.head()

# Merge this dataframe to the school summary dataset "merged2"
merged3_school_summary_df = pd.merge(merged2_school_summary_df, school_num_student_passing_maths_df, on="school_name", how="outer")

# rename the added count to numpassingmaths
merged3_school_summary_df = merged3_school_summary_df.rename(columns={"student_name":"numpassingmaths"})

# calculate the percentage of students passing maths by school and add in as a column
merged3_school_summary_df["% Passing Maths"] = \
        merged3_school_summary_df["numpassingmaths"] / \
        merged3_school_summary_df["size"]*100
#merged3_school_summary_df.head()

In [42]:
# Calculate the number of passing reading by school and convert it to dataframe
students_passing_reading = school_data_merged.loc[school_data_merged["reading_score"]>=50]
students_passing_reading.head()

school_num_student_passing_reading_df = pd.DataFrame(students_passing_reading.groupby("school_name")["student_name"].count())
school_num_student_passing_reading_df.head()

# Merge this dataframe to the school summary dataset merged with maths and reading score
merged4_school_summary_df = pd.merge(merged3_school_summary_df, school_num_student_passing_reading_df, on="school_name", how="outer")
merged4_school_summary_df.head()

# rename the added count to numpassingreading
merged4_school_summary_df = merged4_school_summary_df.rename(columns={"student_name":"numpassingreading"})
merged4_school_summary_df.head()

# calculate the percentage of students passing maths by school and add in as a column
merged4_school_summary_df["% Passing Reading"] = \
        merged4_school_summary_df["numpassingreading"] / \
        merged4_school_summary_df["size"]*100
#merged4_school_summary_df.head()

In [44]:
# Calculate percentage of passing both matchs and reading using groupby
# using the dataset previously created from filtering overall passing students

school_num_student_passing_overall_df = overall_passing_df.groupby("school_name")["student_name"].count()
school_num_student_passing_overall_df.head()

# Merge this dataframe to the school summary dataset merged with maths and reading score
finalmerged_school_summary_df = pd.merge(merged4_school_summary_df, school_num_student_passing_overall_df, on="school_name", how="outer")
finalmerged_school_summary_df.head()

# rename the added count to numpassingmaths
finalmerged_school_summary_df = finalmerged_school_summary_df.rename(columns={"student_name":"numpassingoverall"})

# calculate the percentage of students passing overall by school and add in as a column
finalmerged_school_summary_df["% Overall Passing"] = \
        finalmerged_school_summary_df["numpassingoverall"] / \
        finalmerged_school_summary_df["size"]*100
finalmerged_school_summary_df.head()

,school_name,type,size,budget,Per Student Budget,maths_score,reading_score,numpassingmaths,% Passing Maths,numpassingreading,% Passing Reading,numpassingoverall,% Overall Passing
0,Huang High School,Government,2917,1910635,655.0,68.935207,68.910525,2383,81.693521,2376,81.453548,1946,66.712376
1,Figueroa High School,Government,2949,1884411,639.0,68.698542,69.077993,2408,81.654798,2442,82.807731,1995,67.650051
2,Shelton High School,Independent,1761,1056600,600.0,72.034072,70.257808,1612,91.538898,1527,86.712095,1389,78.875639
3,Hernandez High School,Government,4635,3022020,652.0,68.874865,69.186408,3752,80.949299,3795,81.877023,3076,66.364617
4,Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,1339,91.212534,1299,88.487738,1194,81.335150


In [45]:
# Tidy up the merged file to match the screenshot in the starter script

# delete the columns not required "numbers of students passing" by school
del finalmerged_school_summary_df['numpassingmaths']
del finalmerged_school_summary_df['numpassingreading']
del finalmerged_school_summary_df['numpassingoverall']
#finalmerged_school_summary_df.head()

In [46]:
# change the column names

school_summary_newcols_df = finalmerged_school_summary_df.rename(columns={"type":"School Type", \
                                                                    "size":"Total Students",\
                                                                    "budget":"Total School Budget",\
                                                                    "maths_score":"Average Maths Score",\
                                                                    "reading_score":"Average Reading Score"
                                                                   })

# sort by school name
school_summary_sorted_df = school_summary_newcols_df.sort_values(["school_name"],ascending=True)

# use map to format "total school budget" and "per student budget"
formated_school_summary_sorted_df=school_summary_sorted_df.copy()
formated_school_summary_sorted_df["Total School Budget"] = formated_school_summary_sorted_df["Total School Budget"].map("${:,.2f}".format)
formated_school_summary_sorted_df["Per Student Budget"] = formated_school_summary_sorted_df["Per Student Budget"].map("${:,.2f}".format)
formated_school_summary_sorted_df.head(3)

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
7,Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
6,Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
1,Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051


In [47]:
# Highest-Performing Schools (by % Overall Passing)
## sort by % Overall Passing in descending order and display the top 5 rows
school_summary_sorted_df = school_summary_sorted_df.sort_values(["% Overall Passing"],ascending=False)
#school_summary_sorted_df.head(5)

# Save the top 5 highest performing schools as a DataFrame using iloc

highest_performing_schools_df = school_summary_sorted_df.iloc[0:5, :]
highest_performing_schools_df

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
4,Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
6,Cabrera High School,Independent,1858,1081356,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
7,Bailey High School,Government,4976,3124928,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
10,Wright High School,Independent,1800,1049400,583.0,72.047222,70.969444,91.777778,86.666667,79.722222
11,Rodriguez High School,Government,3999,2547363,637.0,72.047762,70.935984,90.797699,87.396849,79.419855


In [48]:
# Lowest-Performing Schools (by % Overall Passing)
## sort by % Overall Passing in ascending order and display the top 5 rows
school_summary_sorted_df = school_summary_sorted_df.sort_values(["% Overall Passing"],ascending=True)
#school_summary_sorted_df.head(5)

# Save the top 5 highest performing schools as a DataFrame using iloc

lowest_performing_schools_df = school_summary_sorted_df.iloc[0:5, :]
lowest_performing_schools_df

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
3,Hernandez High School,Government,4635,3022020,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
0,Huang High School,Government,2917,1910635,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
12,Johnson High School,Government,4761,3094650,650.0,68.843100,69.039277,82.062592,81.978576,67.191766
5,Wilson High School,Independent,2283,1319574,578.0,69.170828,68.876916,82.785808,81.296540,67.455103
13,Ford High School,Government,2739,1763916,644.0,69.091274,69.572472,82.438846,82.219788,67.469880


In [49]:
# Maths Scores by Year
# Create data series for each year using iloc

year_9 = school_data_merged[school_data_merged.iloc[:,3] == 9]
year_10 = school_data_merged[school_data_merged.iloc[:,3] == 10]
year_11 = school_data_merged[school_data_merged.iloc[:,3] == 11]
year_12 = school_data_merged[school_data_merged.iloc[:,3] == 12]
year_12.head(3)

# Calculate average maths scores by Year
Average_mathsscore_year9_df = year_9.groupby("school_name")["maths_score"].mean()
#Average_mathsscore_year9_df.head()
Average_mathsscore_year10_df = year_10.groupby("school_name")["maths_score"].mean()
#Average_mathsscore_year10_df.head()
Average_mathsscore_year11_df = year_11.groupby("school_name")["maths_score"].mean()
Average_mathsscore_year11_df.head()
Average_mathsscore_year12_df = year_12.groupby("school_name")["maths_score"].mean()
Average_mathsscore_year12_df.head()
# merge the above into one DataFrame
Maths_scores_byyear_df = pd.merge(Average_mathsscore_year9_df, Average_mathsscore_year10_df, on="school_name", how="outer")

CleanedMaths_scores_byyear_df = Maths_scores_byyear_df.rename(columns={"maths_score_x":"Year 9", \
                                                                    "maths_score_y":"Year 10",\
                                                                   })
CleanedMaths_scores_byyear_df = pd.merge(CleanedMaths_scores_byyear_df, Average_mathsscore_year11_df, on="school_name", how="outer")
CleanedMaths_scores_byyear_df = pd.merge(CleanedMaths_scores_byyear_df, Average_mathsscore_year12_df, on="school_name", how="outer")
CleanedMaths_scores_byyear_df = CleanedMaths_scores_byyear_df.rename(columns={"maths_score_x":"Year 11", \
                                                                    "maths_score_y":"Year 12",\
                                                                   })
CleanedMaths_scores_byyear_df 

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393


In [50]:
# Calculate average reading scores by Year
Average_readingscore_year9_df = year_9.groupby("school_name")["reading_score"].mean()
#Average_readingscore_year9_df.head()

Average_readingscore_year10_df = year_10.groupby("school_name")["reading_score"].mean()
#Average_readingscore_year10_df.head()

Average_readingscore_year11_df = year_11.groupby("school_name")["reading_score"].mean()
#Average_readingscore_year11_df.head()

Average_readingscore_year12_df = year_12.groupby("school_name")["reading_score"].mean()
#Average_readingscore_year12_df.head()
# merge the above into one DataFrame
Reading_scores_byyear_df = pd.merge(Average_readingscore_year9_df, Average_readingscore_year10_df, on="school_name", how="outer")

CleanedReading_scores_byyear_df = Reading_scores_byyear_df.rename(columns={"reading_score_x":"Year 9", \
                                                                    "reading_score_y":"Year 10",\
                                                                   })

CleanedReading_scores_byyear_df = pd.merge(CleanedReading_scores_byyear_df, Average_readingscore_year11_df, on="school_name", how="outer")
CleanedReading_scores_byyear_df = pd.merge(CleanedReading_scores_byyear_df, Average_readingscore_year12_df, on="school_name", how="outer")
CleanedReading_scores_byyear_df = CleanedReading_scores_byyear_df.rename(columns={"reading_score_x":"Year 11", \
                                                                    "reading_score_y":"Year 12",\
                                                                   })
CleanedReading_scores_byyear_df 

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521


In [51]:
# Scores by School Spending
#school_summary_sorted_df.columns
#school_summary_sorted_df.dtypes

# Use the code provided to create four bins

bins = [0, 585, 630, 645, 680]
per_student = ["<$585", "$585-630", "$630-645","$645-680"]

# Use pd.cut to categorise spending based on the bins
school_summary_sorted_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_sorted_df["Per Student Budget"], bins, labels=per_student, include_lowest=True)
#school_summary_sorted_df.head()


# Use the code provided to calculate mean scores per spending range
spending_math_scores_df = pd.DataFrame(school_summary_sorted_df.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean())                                       
spending_reading_scores_df = pd.DataFrame(school_summary_sorted_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean())
spending_passing_math_df = pd.DataFrame(school_summary_sorted_df.groupby(["Spending Ranges (Per Student)"])["% Passing Maths"].mean())
spending_passing_reading_df = pd.DataFrame(school_summary_sorted_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean())
overall_passing_spending_df = pd.DataFrame(school_summary_sorted_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean())
overall_passing_spending_df

# Create a DataFrame "spending_summary" by merging these datasets
merged1_spending_summary_df = pd.merge(spending_math_scores_df, spending_reading_scores_df, on="Spending Ranges (Per Student)", how="outer")
merged2_spending_summary_df = pd.merge(merged1_spending_summary_df, spending_passing_math_df, on="Spending Ranges (Per Student)", how="outer")
merged3_spending_summary_df = pd.merge(merged2_spending_summary_df, spending_passing_reading_df, on="Spending Ranges (Per Student)", how="outer")
spending_summary_df = pd.merge(merged3_spending_summary_df, overall_passing_spending_df, on="Spending Ranges (Per Student)", how="outer")

spending_summary_df

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.364587,70.716577,88.835926,86.390517,76.721458
$585-630,72.065868,71.031297,91.518824,87.292423,79.876293
$630-645,69.854807,69.838814,84.686139,83.763585,71.004977
$645-680,68.884391,69.045403,81.568470,81.769716,66.756253


In [52]:
# Scores by School Size

# Use the code provided for binning 
size_bins = [0, 1000, 2000, 5000]
per_size = ["Small(<1000)","Medium(1000-2000)","Large(2000-5000)"]

# Use pd.cut to categorise scores by school size
school_summary_sorted_df["School size"] = pd.cut(school_summary_sorted_df["Total Students"], size_bins, labels=per_size, include_lowest=True)
#school_summary_sorted_df.head()

# Use the code provided to calculate mean scores per school size
persize_math_scores_df = pd.DataFrame(school_summary_sorted_df.groupby(["School size"])["Average Maths Score"].mean())                                       
persize_reading_scores_df = pd.DataFrame(school_summary_sorted_df.groupby(["School size"])["Average Reading Score"].mean())
persize_passing_math_df = pd.DataFrame(school_summary_sorted_df.groupby(["School size"])["% Passing Maths"].mean())
persize_passing_reading_df = pd.DataFrame(school_summary_sorted_df.groupby(["School size"])["% Passing Reading"].mean())
overall_passing_persize_df = pd.DataFrame(school_summary_sorted_df.groupby(["School size"])["% Overall Passing"].mean())
overall_passing_persize_df

# Create a DataFrame "per_school_summary" by merging these datasets
merged1_per_school_summary_df = pd.merge(persize_math_scores_df, persize_reading_scores_df, on="School size", how="outer")
merged2_per_school_summary_df = pd.merge(merged1_per_school_summary_df, persize_passing_math_df, on="School size", how="outer")
merged3_per_school_summary_df = pd.merge(merged2_per_school_summary_df, persize_passing_reading_df, on="School size", how="outer")
per_school_summary_df = pd.merge(merged3_per_school_summary_df, overall_passing_persize_df, on="School size", how="outer")

per_school_summary_df

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School size,,,,,
Small(<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium(1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large(2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


In [53]:
# Use the code provided to calculate mean scores by school type
type_math_scores_df = pd.DataFrame(school_summary_sorted_df.groupby(["School Type"])["Average Maths Score"].mean())                                       
type_reading_scores_df = pd.DataFrame(school_summary_sorted_df.groupby(["School Type"])["Average Reading Score"].mean())
type_passing_math_df = pd.DataFrame(school_summary_sorted_df.groupby(["School Type"])["% Passing Maths"].mean())
type_passing_reading_df = pd.DataFrame(school_summary_sorted_df.groupby(["School Type"])["% Passing Reading"].mean())
overall_passing_pertype_df = pd.DataFrame(school_summary_sorted_df.groupby(["School Type"])["% Overall Passing"].mean())
#overall_passing_pertype_df

# Create a DataFrame "type_summary" by merging these datasets
merged1_type_summary_df = pd.merge(type_math_scores_df, type_reading_scores_df, on="School Type", how="outer")
merged2_type_summary_df = pd.merge(merged1_type_summary_df, type_passing_math_df, on="School Type", how="outer")
merged3_type_summary_df = pd.merge(merged2_type_summary_df, type_passing_reading_df, on="School Type", how="outer")
type_summary_df = pd.merge(merged3_type_summary_df, overall_passing_pertype_df, on="School Type", how="outer")

type_summary_df

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
